# Simple HyDE example

In [2]:

import os

import chromadb
import chromadb.utils.embedding_functions as embedding_functions
from chromadb import Settings
from IPython.display import Markdown, display
from llama_index.core import PromptTemplate, SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from openai import OpenAI, AzureOpenAI

import importlib
import util

# importlib.reload(util.helpers)

In [3]:
from dotenv import load_dotenv

load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")

openai_client = AzureOpenAI(
    api_key=OPENAI_API_KEY,  
    api_version="2024-05-01-preview", # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

In [4]:
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY,
    model_name="text-embedding-ada-002",
    api_type="azure", 
    api_version="2024-05-01-preview"
)

# chroma_client = chromadb.PersistentClient(
#     path="./data/baseline-rag-pdf-docs/chromadb", settings=Settings(allow_reset=True))

In [5]:
import os

from dotenv import load_dotenv
from IPython.display import Markdown, display
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.prompts.default_prompt_selectors import \
    DEFAULT_TREE_SUMMARIZE_PROMPT_SEL
from llama_index.core.query_engine import (RouterQueryEngine,
                                           TransformQueryEngine)
from llama_index.core.response_synthesizers import TreeSummarize
from llama_index.core.selectors import LLMMultiSelector
from llama_index.core.tools import QueryEngineTool
from llama_index.llms.openai import OpenAI
# from llama_index.postprocessor.cohere_rerank import CohereRerank

from util.helpers import create_and_save_wiki_md_files, get_wiki_pages
from util.query_engines import VerboseHyDEQueryTransform, WeatherQueryEngine

In [6]:
import chromadb
from chromadb import Settings
from llama_index.llms.openai import OpenAI
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore

from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

# ChromaDB Vector Store
chroma_client = chromadb.PersistentClient(
    path="./data/baseline-rag-pdf-docs/chromadb", settings=Settings(allow_reset=True))
chroma_client.reset()
collection = chroma_client.get_or_create_collection(
    name="landsforsoeg", metadata={"hnsw:space": "cosine"})
vector_store = ChromaVectorStore(chroma_collection=collection)


llm = AzureOpenAI(
    model="gpt-4",
    deployment_name="gpt4",
    api_key=os.getenv("OPENAI_API_KEY"),  
    # api_version=os.getenv("OPENAI_API_VERSION"),
    api_version = "2024-05-01-preview", # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key=os.getenv("OPENAI_API_KEY"),  
    # api_version=os.getenv("OPENAI_API_VERSION"),
    api_version = "2024-05-01-preview", # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Define the ingestion pipeline to add documents to vector store
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=512, chunk_overlap=20),
        embed_model,
    ],
    vector_store=vector_store,
)

# Create index with the vector store and using the embedding model
# index = VectorStoreIndex.from_vector_store(
#     vector_store=vector_store, embed_model=embed_model)

In [6]:
# Fetch documents
documents = SimpleDirectoryReader('./data/docs').load_data()

# Run pipeline
pipeline.run(documents=documents)

print("Indexing complete")

Indexing complete


In [7]:
index.as_retriever()

In [14]:
hyde = VerboseHyDEQueryTransform(include_original=True, verbose=True)
hyde

In [15]:
transformed_query_engine = TransformQueryEngine(
    query_engine=index.as_query_engine(llm=llm, verbose=True),
    query_transform=hyde,
)

In [16]:
transformed_query_engine

In [165]:
simple_query_engine = index.as_query_engine(similarity_top_k=5)
simple_response = simple_query_engine.query('hvordan bekæmpes væselhale?')
display(Markdown(simple_response.response))

Væselhale kan bekæmpes ved at anvende forskellige herbicider. For eksempel kan Mateno Duo 600 SC anvendes i forskellige mængder, som 0,7 liter pr. hektar. Det er også blevet observeret, at en tidlig sprøjtning, for eksempel den 20. august, kan være effektiv. Tilsætning af Boxer kan også reducere mængden af væselhale i frøet, men det kan ikke anvendes i praksis på de behandlingstidspunkter, der er blevet anvendt i forsøget, på grund af fordampning af prosulfocarb.

In [145]:
chunks = simple_response.source_nodes
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}: {chunk.score}")
    # print(dir(chunk))

Chunk 1: 0.8261254989814707
Chunk 2: 0.8118815719307365
Chunk 3: 0.8082726919197029
Chunk 4: 0.8074034576788202
Chunk 5: 0.8048558014178372


# RAG implementation using HyDE

In [7]:
query = "hvordan bekæmper jeg væselhale?"

In [18]:
llm = AzureOpenAI(
    model="gpt-4",
    deployment_name="gpt4",
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"), # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"), # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

In [8]:
documents = SimpleDirectoryReader('./data/docs').load_data()

from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

# build VectorStoreIndex that takes care of chunking documents
# and encoding chunks to embeddings for future retrieval
# index = VectorStoreIndex.from_documents(documents=documents)#, 
                                        #api_key=os.environ['OPENAI_API_KEY'],
                                        #base_url=os.environ['AZURE_API_BASE'],
                                        #app_url=os.environ['AZURE_OPENAI_ENDPOINT'])
index = VectorStoreIndex.from_documents(documents=documents, embed_model=embed_model, llm=llm, verbose=True)

hyde = VerboseHyDEQueryTransform(include_original=True, verbose=True)

hyde_query_engine = TransformQueryEngine(
    query_engine=index.as_query_engine(llm=llm, verbose=True, streaming=True, similarity_top_k=5),
    query_transform=hyde,
)


hyde_response = hyde_query_engine.query(query)
display(Markdown(f"{hyde_response}"))

<b>[VerboseHyDEQueryTransform]<b> Generated hypothetical document: Væselhale er en ukrudtsplante, der kan være svær at bekæmpe, men det er ikke umuligt. Først og fremmest er det vigtigt at forstå, at væselhale er en årlig plante, der formerer sig gennem frø. Derfor er det vigtigt at bekæmpe planten, før den når at sætte frø.

En effektiv metode til at bekæmpe væselhale er at bruge en ukrudtsbrænder. Denne metode er mest effektiv, hvis du bruger den i det tidlige forår, når væselhale begynder at vokse. Ukrudtsbrænderen vil dræbe planten og forhindre den i at sætte frø.

En anden metode er at bruge en ukrudtsmiddel, der er specifikt designet til at bekæmpe væselhale. Disse produkter kan købes i de fleste havecentre og online. Følg altid instruktionerne på produktet nøje for at sikre, at det er effektivt.

Endelig kan du også bekæmpe væselhale ved at luge den ud manuelt. Dette kan være en tidskrævende proces, men det er en effektiv metode, hvis du kun har et lille område med væselhale. Sørg for at få hele planten med, inklusive rødderne, for at forhindre den i at vokse tilbage.

Husk, at det er vigtigt at bekæmpe væselhale så tidligt som muligt for at forhindre den i at sprede sig. Med tålmodighed og vedholdenhed kan du effektivt bekæmpe denne ukrudtsplante.

-------------------



Den bedste bekæmpelse af væselhale er opnået ved at anvende 0,7 l. Mateno Duo 600 SC pr. ha den 20. august, hvilket betyder en tidlig sprøjtning. Tilsætning af Boxer har i et forsøg resulteret i færre væselhaler i frøet, men kan ikke anvendes i praksis på de behandlingstidspunkter, der har været anvendt i dette forsøg, på grund af fordampning af prosulfocarb.

In [146]:
hyde_response = hyde_query_engine.query(query)
display(Markdown(f"{hyde_response}"))

<b>[VerboseHyDEQueryTransform]<b> Generated hypothetical document: Væselhale er en ukrudtsplante, der kan være svær at bekæmpe. Her er nogle trin, du kan tage for at bekæmpe væselhale:

1. Fjernelse: Det første skridt er at fjerne så mange planter som muligt. Dette kan gøres manuelt ved at trække dem op med rødderne. Det er vigtigt at gøre dette, før planten har haft mulighed for at sprede sine frø.

2. Brug af herbicider: Hvis manuel fjernelse ikke er nok, kan du overveje at bruge et herbicid. Det er vigtigt at vælge et herbicid, der er specifikt designet til at bekæmpe væselhale, da nogle almindelige herbicider ikke er effektive mod denne plante.

3. Forebyggelse: Efter at have fjernet de eksisterende planter, er det vigtigt at tage skridt til at forhindre, at de vender tilbage. Dette kan omfatte regelmæssig klipning af græsset for at forhindre, at væselhale får mulighed for at vokse, og brug af en præ-emergent herbicid for at forhindre, at nye frø spirer.

4. Overvågning: Endelig er det vigtigt at overvåge området regelmæssigt for tegn på ny vækst. Hvis du ser nye planter, skal du fjerne dem så hurtigt som muligt for at forhindre, at de spreder sig.

Ved at følge disse trin kan du effektivt bekæmpe væselhale i din have eller på dit græsareal.

-------------------



Den bedste bekæmpelse af væselhale er opnået ved at anvende 0,7 l. Mateno Duo 600 SC pr. ha den 20. august, hvilket betyder en tidlig sprøjtning. Tilsætning af Boxer kan også reducere mængden af væselhale i frøet, men det kan ikke anvendes i praksis på de behandlingstidspunkter, der er blevet anvendt i forsøget, på grund af fordampning af prosulfocarb.

In [20]:
# response = transformed_query_engine.query(query)
chunks = hyde_response.source_nodes
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}: {chunk.score}")
    # print(dir(chunk))

Chunk 1: 0.8566892893911661
Chunk 2: 0.843333123914906
Chunk 3: 0.8429606370235365
Chunk 4: 0.842242481445407
Chunk 5: 0.8416756075072966


In [147]:
prompt = """You are a helpful assistant that answers questions for farmers and consultants for farmers using provided context.
    Context information is below.
    ---------------------
    {context}
    ---------------------
    Given the context information and not prior knowledge, answer the query. Always provide an answer in the danish language.
    Below the answer, the source of the answer should be provided including file_name, chapter and which section the source is found.
    Query: {query}
    Answer: 
    """

transformed_prompt_query_engine = TransformQueryEngine(
    query_engine=index.as_query_engine(llm=llm, verbose=True, streaming=True, similarity_top_k=5, PromptTemplate=prompt),
    query_transform=hyde,
)

In [148]:
# response = transformed_query_engine.query('hvordan vælger jeg den bedste vårbygsort?')
transformed_prompt_response = transformed_prompt_query_engine.query('hvordan bekæmper jeg væselhale?')
display(Markdown(f"{transformed_prompt_response}"))

<b>[VerboseHyDEQueryTransform]<b> Generated hypothetical document: Væselhale er en ukrudtsplante, der kan være svær at bekæmpe, men det er ikke umuligt. Her er nogle trin, du kan tage for at bekæmpe væselhale:

1. Fysisk fjernelse: Den mest direkte metode til at bekæmpe væselhale er at fjerne den fysisk. Dette kan gøres ved at grave planten op med rødderne. Det er vigtigt at få hele roden, da planten kan vokse tilbage, hvis nogen del af roden forbliver i jorden.

2. Brug af herbicider: Der er flere herbicider på markedet, der er effektive til at bekæmpe væselhale. Det er vigtigt at følge instruktionerne nøje for at sikre, at du bruger produktet sikkert og effektivt.

3. Forebyggelse: En af de bedste måder at bekæmpe væselhale på er at forhindre, at det vokser i første omgang. Dette kan gøres ved at holde din have eller græsplæne sund og velplejet. En stærk, sund græsplæne kan hjælpe med at forhindre vækst af ukrudt som væselhale.

4. Professionel hjælp: Hvis du har en stor infestation af væselhale, kan det være nødvendigt at søge professionel hjælp. En professionel gartner eller landskabsplejer vil have adgang til mere kraftfulde værktøjer og teknikker til at bekæmpe væselhale.

Husk, at bekæmpelse af væselhale kan tage tid, og det kan være nødvendigt at gentage disse trin flere gange for at fuldstændigt fjerne planten fra din have eller græsplæne.

-------------------



Den mest effektive bekæmpelse af væselhale er opnået ved at anvende 0,7 l. Mateno Duo 600 SC pr. ha den 20. august, hvilket betyder en tidlig sprøjtning. Tilføjelse af Boxer i denne proces har resulteret i færre væselhaler i frøet, men kan ikke anvendes i praksis på de behandlingstidspunkter, der er blevet brugt i dette forsøg på grund af fordampning af prosulfocarb. Det er vigtigt at bemærke, at selvom denne metode har reduceret mængden af væselhale i frøet betydeligt sammenlignet med ubehandlet, er indholdet stadig for højt i forhold til certificering.

In [23]:
# response = transformed_query_engine.query(query)
chunks = transformed_prompt_response.source_nodes
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}: {chunk.score}")
    # print(dir(chunk))

Chunk 1: 0.8534335892857363
Chunk 2: 0.839870752577824
Chunk 3: 0.839793748239762
Chunk 4: 0.8376136905314411
Chunk 5: 0.8375681202299005


# Reranking

In [158]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import QueryBundle
from llama_index.postprocessor.rankgpt_rerank import RankGPTRerank

import pandas as pd
from IPython.display import display, HTML

def get_retrieved_nodes(query_str, vector_top_k=10, reranker_top_n=3, with_reranker=False): 
    query_bundle = QueryBundle(query_str)
    # configure retriever
    retriever = VectorIndexRetriever(
        index=index,
        similarity_top_k=vector_top_k
    )
    retrieved_nodes = retriever.retrieve(query_bundle)

    if with_reranker:
        # configure reranker
        reranker = RankGPTRerank(
            llm=llm,
            top_n=reranker_top_n,
            verbose=True
        )
        retrieved_nodes = reranker.postprocess_nodes(
            retrieved_nodes, query_bundle
        )
    return retrieved_nodes

def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n", "<br>")))


def visualize_retrieved_nodes(nodes) -> None:
    result_dicts = []
    for node in nodes:
        result_dict = {"Score": node.score, "Text": node.node.get_text()}
        result_dicts.append(result_dict)

    pretty_print(pd.DataFrame(result_dicts))

In [162]:
new_nodes = get_retrieved_nodes(
    "Hvordan bekæmper jeg væselhale",
    vector_top_k=10,
    reranker_top_n=3,
    with_reranker=True
)

After Reranking, new rank list for nodes: [0, 1, 5, 9, 3, 2, 8, 7, 6, 4]

In [163]:
new_nodes

[NodeWithScore(node=TextNode(id_='bdc801d8-b325-48a0-af29-0eed1fe55000', embedding=None, metadata={'page_label': '128', 'file_name': 'planter_landsforsogene_2022.pdf', 'file_path': '/home/jaav/proj/RAG/advanced-rag-examples/data/docs/planter_landsforsogene_2022.pdf', 'file_type': 'application/pdf', 'file_size': 53911931, 'creation_date': '2024-08-05', 'last_modified_date': '2024-07-30'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='60a6243b-ba90-4cf0-b09e-f5b8fa1b1e0f', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '128', 'file_name': 'planter_landsforsogene_2022.pdf', 'file_path': '/home/jaav/proj/RAG/advanced-rag-examples/data/docs/planter_landsforsogene_2022.pdf', 'file_type': 'applica

In [164]:
visualize_retrieved_nodes(new_nodes)

,Score,Text
0,0.820186,"Den bed -ste bekæmpelse af væselhale er opnået i led 3, hvor der har været anvendt 0,7 l. Mateno Duo 600 SC pr. ha den 20. august – altså en tidlig sprøjtning. Indholdet af væ -selhale i frøet i led 3 er dog stadig for højt i forhold til certificering, men er nedbragt væsentligt sammenlignet med ubehandlet. Tilsætning af Boxer giver i dette forsøg færre væselhaler i frøet, men kan grundet fordampning af prosulfocarb ikke anvendes i praksis på de behand -lingstidspunkter, der har været anvendt i dette forsøg. Der har været en god bekæmpelse af alm. rapgræs i Slemning efter kraftig nedbør umiddelbart efter såning giver udfordringer med fremspiringen.FOTO: KRISTIAN JURANICH, SEGES INNOVATION"
1,0.812187,"Angrebene ligger i 2022 på niveau med 2020, hvor der også var en tør periode i foråret (medio marts-ultimo april).Meldug er kommet relativt sent, og angrebene har over -vejende været svage til moderate. I enkeltmarker har dog været mere udbredte angreb .TABEL 18. Bekæmpelse af væselhale i vinterhvede om efter -året. (E16)Vinterhvede StadieVæselhaleKemi og udbring-ning,kr. pr. haOktober NovemberAntal planter pr. m2Antal planter pr. m2Procent effekt2021-22, 3 forsøg1.Ubehandlet - 602 a 632 a 0 a 2.1,5 l Boxer + 0,05 l DFF 10-11 57 bc 41 b 92 b 3603.0,7 l Mateno Duo + 1 l Boxer 10-11 9 de 4 c 99 de 5424.0,5 l Mateno Duo + 1 l Boxer 10-11 6 de 2 cd 99 de 4595.0,35 l Mateno Duo + 1,5 l Boxer 10-11 10 de 5 c 98 bcd 4866.0,7 l Mateno Duo 10-11 69 bc 61 b 92 bc 3627.0,5 l Mateno Duo + 1 l Boxer og0,75 l Atlantis OD10-1112 - 0 d 99 e 7758.0,75 l Atlantis OD 12 - 174 ab 58 a 3179.0,7 l Mateno Duo + 1 l Boxer 12 - 6 c 98 cd 542Resultater med forskellige bogstaver er signifikant forskellige.Optimalt tidspunkt for bekæmpelse af væselhale med Boxer og Mateno Duo.FOTO: SOFIE HÆSTRUP OLESEN, LANDBONORD"
2,0.803691,"Forsøget er anlagt i et design, der gør at alle behand -linger, udover etableringen, er de samme som den om -TABEL 10. Bekæmpelse af væselhale i rødsvingel (J10)RødsvingelBehandlings-tidspunktHerbicid-skade1)3/4Væselhale pct. dækning af jord 3/4Væselhale pct. i frø ved høstUdb. og merudb., kg frø pr. haNettomer-udbytte, kg pr. ha2022. 2 forsøg1.1. Ubehandlet, 0,0 3,4 0,15 1467 -2.0,35 l Mateno Duo 600 SC 20/8 2021 0,0 1,9 0,10 49 293.0,7 l Mateno Duo 600 SC 20/8 2021 0,3 2,4 0,00 72 394.0,35 l Mateno Duo 600 SC 15/9 2021 0,0 1,9 0,10 5 -145.0,7 l Mateno Duo 600 SC 15/9 2021 0,3 2,4 0,10 -17 -506.0,35 l Mateno Duo 600 SC0,35 l Mateno Duo 600 SC20/8 202115/9 2021 0,4 3,4 0,05 24 -167.0,35 l Mateno Duo 600 SC + 1 l Boxer 20/8 2021 0,1 3,8 0,05 -22 -588.0,35 l Mateno Duo 600 SC + 1 l Boxer 15/9 2021 0,0 2,4 0,10 65 299.0,35 l Mateno Duo 600 SC + 0,5 l Boxer0,35 l Mateno Duo 600 SC + 0,5 l Boxer20/8 202115/9 2021 1,1 2,7 0,05 -18 -7410. 0,35 l Mateno Duo 600 SC + 1 l Boxer0,35 l Mateno Duo 600 SC + 1 l Boxer20/8 202115/9 2021 1,1 2,6 0,05 -4 -76LSD ns1) Karakter for herbicidskade, 0 = ingen skade og 10 = mest skadeFIGUR 3. Fordeling af frø set på tværs af såmaskinerne.40100160135791113151719212325Indekstal, 100 er gennemsnitUdløbsnummer, set bagfra såmaskinenFordeling af græsfrøudsæd på såudløbene Såmaskine 1 Såmaskine 2 Såmaskine 3Variationskoefficient (relativ spredning af et datasæt 0- 100%): 8% 15% 24%Etablering af forsøg med test af såmaskiner til frø på en flot dag i marts.FOTO: KRISTIAN JURANICH, SEGES INNOVATION"


# Step-Back prompting

In [57]:
from util.query_engines import VerboseHyDEQueryTransform, VerboseStepBackQueryEngine, RewriteRetrieveReadQueryEngine
from typing import Optional

from llama_index.llms.openai import OpenAI
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    PromptTemplate,
    Settings,
)
from llama_index.core.prompts.prompt_type import PromptType
from llama_index.core.prompts.base import BasePromptTemplate
from llama_index.core.prompts.mixin import PromptDictType
from llama_index.core.query_engine import CustomQueryEngine, TransformQueryEngine
from llama_index.core.schema import QueryBundle
from llama_index.core.indices.query.query_transform.base import BaseQueryTransform
from llama_index.core.service_context_elements.llm_predictor import LLMPredictorType

In [130]:
# documents = SimpleDirectoryReader('./data/docs').load_data()
# index = VectorStoreIndex.from_documents(documents=documents)
step_back_query_engine = VerboseStepBackQueryEngine(retriever=index.as_retriever(similarity_top_k=5), llm=llm, verbose=True)

step_back_response = step_back_query_engine.query("hvordan bekæmper jeg væselhale?")
display(Markdown(step_back_response['response']))



Step-back Question:

 What are some general strategies for dealing with pests? 




Step-back Context:

 [NodeWithScore(node=TextNode(id_='7d9404fb-05a1-4176-bbd5-c1bc15ee2089', embedding=None, metadata={'page_label': '58', 'file_name': 'planter_landsforsogene_2023.pdf', 'file_path': '/home/jaav/proj/RAG/advanced-rag-examples/data/docs/planter_landsforsogene_2023.pdf', 'file_type': 'application/pdf', 'file_size': 47095474, 'creation_date': '2024-08-05', 'last_modified_date': '2024-07-30'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='4c7b2bde-e78a-4406-8404-263fa22fe005', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '58', 'file_name': 'planter_landsforsogene_2023.pdf', 'file_path': '/h

Bekæmpelse af væselhale kan være en udfordring, da det er en robust ukrudtsart. Men der er flere metoder, der kan anvendes:

1. Kemisk bekæmpelse: I nogle tilfælde kan kemiske herbicider anvendes til at bekæmpe væselhale. I en undersøgelse blev der anvendt forskellige kombinationer af herbicider, herunder Mateno Duo og Boxer, som viste gode resultater i bekæmpelsen af væselhale. Det er dog vigtigt at bemærke, at kemisk bekæmpelse ikke kan stå alene, og at det er nødvendigt at bringe bestanden ned på et niveau, hvor de tilbageværende væselhale efter kemisk bekæmpelse er få og uden betydning for fortsat opformering og spredning.

2. Tidlig sprøjtning: Tidlig sprøjtning kan også være effektiv i bekæmpelsen af væselhale. I undersøgelsen blev der opnået bedst bekæmpelse ved tidlig sprøjtning den 20. august.

3. Falsk såbed: En anden metode er at skabe et falsk såbed, hvor man lader væselhale spire og derefter ødelægger det ved at harve jorden, inden man sår afgrøden. Dette kan reducere mængden af væselhale, der spirer sammen med afgrøden.

4. Konkurrencedygtige afgrøder: At så konkurrencedygtige afgrøder kan også hjælpe med at bekæmpe væselhale, da de kan konkurrere om lys, vand og næringsstoffer.

5. Anvendelse af IPM-strategier: Integreret plantebeskyttelse (IPM) strategier, der kombinerer forskellige metoder til bekæmpelse af ukrudt, kan også være effektive. Dette kan omfatte en kombination af kemisk bekæmpelse, falsk såbed, konkurrencedygtige afgrøder og andre metoder.

Det er vigtigt at bemærke, at effektiviteten af disse metoder kan variere afhængigt af specifikke dyrkningsforhold og væselhalepopulationer. Derfor kan det være nødvendigt at tilpasse strategierne til de specifikke forhold på din ejendom.

In [34]:
normal_context = step_back_response['normal_context']

for node_with_score in normal_context:
    score = node_with_score.score
    # text = node_with_score.node.text
    print(f"Score: {score}")
    # print(f"Text: {text}")

Score: 0.8261254989814707
Score: 0.8118815719307365
Score: 0.8082726919197029
Score: 0.8074034576788202
Score: 0.8048558014178372


In [35]:
step_back_context = step_back_response['step_back_context']

for node_with_score in step_back_context:
    score = node_with_score.score
    print(f"Score: {score}")

Score: 0.7926054091929144
Score: 0.7892443182476586
Score: 0.7871778928369497
Score: 0.7864525442119366
Score: 0.7857837701026179


# Simple Hybrid Search

In [9]:
from llama_index.core import Settings

nodes = Settings.node_parser.get_nodes_from_documents(documents)

In [10]:
from llama_index.core import StorageContext

# initialize storage context (by default it's in-memory)
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

### Defining vector index and keyword index over the same data

In [11]:
from llama_index.core import SimpleKeywordTableIndex, VectorStoreIndex

vector_index = VectorStoreIndex(nodes, storage_context=storage_context)
keyword_index = SimpleKeywordTableIndex(nodes, storage_context=storage_context)

In [39]:
elements = vector_index._get_node_with_embedding

print(elements)

<bound method VectorStoreIndex._get_node_with_embedding of <llama_index.core.indices.vector_store.base.VectorStoreIndex object at 0x7f381aaae990>>


### Defining custom retriever

In [102]:
# import QueryBundle
from llama_index.core import QueryBundle
from llama_index.core.query_engine import RetrieverQueryEngine

# import NodeWithScore
from llama_index.core.schema import NodeWithScore

# Retrievers
from llama_index.core.retrievers import (
    BaseRetriever,
    VectorIndexRetriever,
    KeywordTableSimpleRetriever,
    # KeywordTableGPTRetriever,
)

from typing import List
from langchain_community.retrievers import BM25Retriever

In [79]:
class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both semantic search and hybrid search."""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        keyword_retriever: KeywordTableSimpleRetriever,
        mode: str = "AND",
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever
        self._keyword_retriever = keyword_retriever
        if mode not in ("AND", "OR"):
            raise ValueError("Invalid mode.")
        self._mode = mode
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        keyword_nodes = self._keyword_retriever.retrieve(query_bundle)

        vector_ids = {n.node.node_id for n in vector_nodes}
        keyword_ids = {n.node.node_id for n in keyword_nodes}

        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in keyword_nodes})

        if self._mode == "AND":
            retrieve_ids = vector_ids.intersection(keyword_ids)
        else:
            retrieve_ids = vector_ids.union(keyword_ids)

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes
        # return{
        #     "retrieve_nodes": retrieve_nodes,
        #     "vector_nodes": vector_nodes,
        #     "keyword_nodes": keyword_nodes
        # }

In [109]:
class LLMQueryEngine(RetrieverQueryEngine):
    def __init__(self,retriever,llm):
        super().__init__(retriever=retriever, response_synthesizer=None)
        self.llm = llm
    
    def query(self, query_str: str):
        # Retrieve contexts
        retrieved_contexts = self.retriever.retrieve(query_str)

        # retrieved_nodes = retrieved_contexts['retrieve_nodes']

        # vector_nodes = retrieved_contexts['vector_nodes']

        # keyword_nodes = retrieved_contexts['keyword_nodes']

        # Combine retrieved contexts into a single prompt
        combined_context = "\n".join([context.node.get_content() for context in retrieved_contexts])
        prompt = f"Query: {query_str}\nContext: {combined_context}\n Response:"

        # Generate response using the LLM
        response = self.llm.complete(prompt)

        return response
        # return{
        #     "response": str(response),
        #     "context": combined_context,
        #     "vector_nodes": vector_nodes,
        #     "keyword_nodes": keyword_nodes
        # }

In [99]:
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

# define custom retriever
vector_retriever = VectorIndexRetriever(index=vector_index, similarity_top_k=2)
keyword_retriever = KeywordTableSimpleRetriever(index=keyword_index)
custom_retriever = CustomRetriever(vector_retriever, keyword_retriever, mode='OR')

# define response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
custom_query_engine = RetrieverQueryEngine(
    retriever=custom_retriever,
    response_synthesizer=response_synthesizer,
)

# vector query engine
vector_query_engine = RetrieverQueryEngine(
    retriever=vector_retriever,
    response_synthesizer=response_synthesizer,
)
# keyword query engine
keyword_query_engine = RetrieverQueryEngine(
    retriever=keyword_retriever,
    response_synthesizer=response_synthesizer,
)

In [104]:
retrieved_chunks = keyword_retriever.retrieve("hvordan bekæmper jeg væselhale?")
print(retrieved_chunks)

[NodeWithScore(node=TextNode(id_='04a870e0-26e0-4798-9b17-d16870e12a51', embedding=None, metadata={'page_label': '134', 'file_name': 'planter_landsforsogene_2022.pdf', 'file_path': '/home/jaav/proj/RAG/advanced-rag-examples/data/docs/planter_landsforsogene_2022.pdf', 'file_type': 'application/pdf', 'file_size': 53911931, 'creation_date': '2024-08-05', 'last_modified_date': '2024-07-30'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b8f9a4e8-d1b4-4614-b42b-295f2e6c4b17', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '134', 'file_name': 'planter_landsforsogene_2022.pdf', 'file_path': '/home/jaav/proj/RAG/advanced-rag-examples/data/docs/planter_landsforsogene_2022.pdf', 'file_type': 'applica

In [105]:
keywords = keyword_retriever._get_keywords("hvordan bekæmper jeg væselhale?")
print(keywords)

['bekæmper', 'jeg', 'hvordan', 'væselhale']


In [122]:
keyword_response = keyword_query_engine.query("hvordan bekæmper jeg væselhale?")
display(Markdown(keyword_response.response))

For at bekæmpe væselhale, kan du anvende Mateno Duo 600 SC. Det er vigtigt at behandle så tidligt som muligt, for eksempel kort tid efter høst af dæksæd. For at opnå den bedste effekt af Mateno Duo 600 SC, er det afgørende, at der er en vis jordfugt til stede. I nogle tilfælde kan kombinationen af Mateno Duo 600 SC med Boxer give en bedre bekæmpelse af væselhale. Dog kan anvendelsen af Boxer være problematisk på det tidlige tidspunkt på grund af fordampningsproblematikken omkring prosulfocarb.

In [123]:
vector_response = vector_query_engine.query("hvordan bekæmper jeg væselhale?")
display(Markdown(vector_response.response))

For at bekæmpe væselhale kan du anvende 0,7 l. Mateno Duo 600 SC pr. ha. Dette har vist sig at være mest effektivt, når det anvendes tidligt, for eksempel den 20. august. Tilsætning af Boxer kan også reducere mængden af væselhale i frøet, men det kan ikke anvendes i praksis på de behandlingstidspunkter, der er blevet anvendt i dette forsøg, på grund af fordampning af prosulfocarb. Andre muligheder inkluderer brugen af 1,5 l Boxer + 0,05 l DFF, 0,7 l Mateno Duo + 1 l Boxer, 0,5 l Mateno Duo + 1 l Boxer, 0,35 l Mateno Duo + 1,5 l Boxer, 0,7 l Mateno Duo, 0,5 l Mateno Duo + 1 l Boxer og 0,75 l Atlantis OD, eller 0,7 l Mateno Duo + 1 l Boxer. Det optimale tidspunkt for bekæmpelse af væselhale med Boxer og Mateno Duo er ikke specificeret.

In [124]:
custom_response = custom_query_engine.query('Hvordan bekæmper jeg væselhale?')
display(Markdown(custom_response.response))

For at bekæmpe væselhale effektivt, kan du anvende Mateno Duo 600 SC. Den bedste bekæmpelse er opnået ved at anvende 0,7 l. Mateno Duo 600 SC pr. ha den 20. august, hvilket betyder en tidlig sprøjtning. Tilsætning af Boxer kan også give færre væselhaler i frøet, men det kan være problematisk at anvende på det tidlige tidspunkt på grund af fordampningsproblemer med prosulfocarb. Det er vigtigt at bemærke, at væselhale skal bekæmpes så tidligt som muligt for at opnå den bedste effekt.

In [110]:
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

# define custom retriever
vector_retriever = VectorIndexRetriever(index=vector_index, similarity_top_k=2)
keyword_retriever = KeywordTableSimpleRetriever(index=keyword_index)
custom_retriever = CustomRetriever(vector_retriever, keyword_retriever, mode='OR')

# define response synthesizer
llm = AzureOpenAI(
    model='gpt-4',
    deployment_name="gpt4",
    api_key=os.getenv("OPENAI_API_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# assemble query engine
hybrid_query_engine = LLMQueryEngine(
    retriever=custom_retriever,
    llm=llm,
)

In [111]:
hybrid_query_engine

In [112]:
hybrid_response = hybrid_query_engine.query(
    "Hvordan bekæmper jeg væselhale?"
)

In [127]:
display(Markdown(hybrid_response.text))

For at bekæmpe væselhale kan du bruge Mateno Duo 600 SC. Det er vigtigt at sprøjte så tidligt som muligt, helst kort tid efter høst af dæksæd. For at opnå den bedste effekt af Mateno Duo 600 SC, skal der være en vis jordfugt til stede. Tilsætning af Boxer kan give en bedre bekæmpelse af væselhale, men det kan være problematisk at anvende på et tidligt tidspunkt på grund af fordampningsproblemer med prosulfocarb. Det er også vigtigt at bemærke, at kemisk bekæmpelse af væselhale ikke kan stå alene, men at bestanden skal bringes ned på et niveau, hvor de tilbageværende væselhale efter kemisk bekæmpelse er få og uden betydning for fortsat opformering og spredning.